Authors : Jinsu Kim, JunHo Park

ⓒ 2022 CCNets, Inc. All Rights Reserved.

![](https://storage.googleapis.com/kaggle-datasets-images/312121/636393/a5097396fc07cf882d3e0d631b100a36/dataset-cover.jpg?t=2019-08-23-15-00-53)

***

<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 1. Content</i></h1>

<br>

> <h4 style = 'font-family: Times New Roman'>
The Dataset is fully dedicated for the developers who want to train the model on Weather Forecasting for Indian climate.<br><br> This dataset provides data from 1st January 2013 to 24th April 2017 in the city of Delhi, India. <br><br>The 4 parameters here are
meantemp, humidity, wind_speed, meanpressure.
    
  
<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 2. About Dataset</i></h1>

<br> 
    
5 columns in the dataset, below is <u>description of features</u>:

*  (1) <b>meantemp</b>: Mean temperature averaged out from multiple 3 hour intervals in a day. 
    
*  (2) <b>humidity</b>: Humidity value for the day (units are grams of water vapor per cubic meter volume of air).
    
    
*  (3) <b>wind_speed</b>: Wind speed measured in kmph.
    
    
*  (4) <b>meanpressure</b>: Pressure reading of weather (measure in atm)
    
    
<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 3. Goal of the Notebook</i></h1>
    
> <h4 style = 'font-family: Times New Roman'>
Goal is to train and test model using GPT, Pytorch. <br><br>
    The Target Column used here is <b>meantemp</b>
</h4>

 

https://www.kaggle.com/datasets/sumanthvrao/daily-climate-time-series-data

***

<a id="1"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #42c2f5'>1.</b> Import Necessary Libraries </b></h1>

In [1]:
import os
import sys
path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.

In [2]:
import pandas  as pd
train_df = pd.read_csv(path_append + '../data/Daily Climate/DailyDelhiClimateTrain.csv')
test_df = pd.read_csv(path_append + '../data/Daily Climate/DailyDelhiClimateTest.csv')
train_df.head()

,date,meantemp,humidity,wind_speed,meanpressure
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000


<a id="2"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #4290f5'>2.</b> Modeling: Preprocess </b></h1>

In [3]:
print('Train set \n\n')
train_df.head()

Train set 




,date,meantemp,humidity,wind_speed,meanpressure
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000


In [4]:
from datetime import datetime
import pandas as pd

# extract the year, month and day from the date column and drop the date column
def extract_date_parts(date):
    date_obj = datetime.strptime(date, '%Y-%m-%d')
    return date_obj.year, date_obj.month, date_obj.day

train_df[['year', 'month', 'day']] = train_df['date'].apply(
    lambda x: pd.Series(extract_date_parts(x))
)
test_df[['year', 'month', 'day']] = test_df['date'].apply(
    lambda x: pd.Series(extract_date_parts(x))
)

train_df.drop('date', axis=1, inplace=True)
test_df.drop('date', axis=1, inplace=True)

# Convert the year column to categorical
train_df['year'] = train_df['year'].astype('category').cat.codes
test_df['year'] = test_df['year'].astype('category').cat.codes

print(train_df.head())
print(test_df.head())

    meantemp   humidity  wind_speed  meanpressure  year  month  day
0  10.000000  84.500000    0.000000   1015.666667     0      1    1
1   7.400000  92.000000    2.980000   1017.800000     0      1    2
2   7.166667  87.000000    4.633333   1018.666667     0      1    3
3   8.666667  71.333333    1.233333   1017.166667     0      1    4
4   6.000000  86.833333    3.700000   1016.500000     0      1    5
    meantemp   humidity  wind_speed  meanpressure  year  month  day
0  15.913043  85.869565    2.743478     59.000000     0      1    1
1  18.500000  77.222222    2.894444   1018.277778     0      1    2
2  17.111111  81.888889    4.016667   1018.333333     0      1    3
3  18.700000  70.050000    4.545000   1015.700000     0      1    4
4  18.388889  74.944444    3.300000   1014.333333     0      1    5


In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, RobustScaler

# Assuming train_df and test_df are pandas DataFrames

# Initialize scalers
robust_scaler = RobustScaler()
min_max_scaler = MinMaxScaler(feature_range = (0, 1))

# Apply StandardScaler to all but the last column
train_df.iloc[:, 1:-3] = robust_scaler.fit_transform(train_df.iloc[:, 1:-3].astype(float))
test_df.iloc[:, 1:-3] = robust_scaler.transform(test_df.iloc[:, 1:-3].astype(float))

# Ensure the last three columns are converted to float
train_df.iloc[:, -3] = train_df.iloc[:, -3].astype(float)
test_df.iloc[:, -3] = test_df.iloc[:, -3].astype(float)

# Apply MinMaxScaler to the last column
train_df.iloc[:, :1] =  min_max_scaler.fit_transform(train_df.iloc[:, :1].values.reshape(-1, 1))
test_df.iloc[:, :1] = min_max_scaler.transform(test_df.iloc[:, :1].values.reshape(-1, 1))

# Verification of the number of features
num_features = train_df.shape[1]


C:\Users\CCN-team\AppData\Local\Temp\ipykernel_29700\671890747.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1457    3.0
1458    3.0
1459    3.0
1460    3.0
1461    4.0
Name: year, Length: 1462, dtype: float64' has dtype incompatible with int8, please explicitly cast to a compatible dtype first.
  train_df.iloc[:, -3] = train_df.iloc[:, -3].astype(float)
C:\Users\CCN-team\AppData\Local\Temp\ipykernel_29700\671890747.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
109    0.0
110    0.0
111    0.0
112    0.0
113    0.0
Name: year, Length: 114, dtype: float64' has dtype incompatible with int8, please explicitly cast to a compatible dtype first.
  test_df.iloc[:, -3] = test_df.iloc[:, -3].astype(float

In [6]:
train_df

,meantemp,humidity,wind_speed,meanpressure,year,month,day
0,0.122271,1.001431,-1.079544,0.531494,0.0,1,1
1,0.042795,1.344778,-0.562473,0.691120,0.0,1,2
2,0.035662,1.115880,-0.275598,0.755969,0.0,1,3
3,0.081514,0.398665,-0.865544,0.643731,0.0,1,4
4,0.000000,1.108250,-0.437544,0.593848,0.0,1,5
...,...,...,...,...,...,...,...
1457,0.342890,0.248056,-0.463948,0.523903,3.0,12,28
1458,0.282387,1.155120,-0.038462,0.624134,3.0,12,29
1459,0.247453,1.237959,0.007808,0.698959,3.0,12,30
1460,0.276718,1.115880,0.191443,0.563918,3.0,12,31


In [7]:
import torch
from torch.utils.data import Dataset
import random

class SequentialDataset(Dataset):
    def __init__(self, df, indices, max_window_size):
        self.df = df
        self.indices = indices
        self.max_window_size = max_window_size
        self.min_window_size = max_window_size // 2

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        start_idx = self.indices[idx]
        # window_size = random.randint(self.min_window_size, self.max_window_size)
        window_size = self.max_window_size
        end_idx = min(start_idx + window_size, len(self.df))

        seq = self.df.iloc[start_idx:end_idx]
        X, y = seq.values[:-1, :], seq.values[1:, :1] # mean temperature is the last column

        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        return X, y

In [8]:
from random import shuffle

# Assume 'df' is your DataFrame and 'event' is the column containing labels

def generate_indices(df, max_window_size):
    indices = []
    max_index = len(df) - max_window_size + 1  # Calculate the maximum starting index
    
    for i in range(max_index):
        # Check if all labels in the window are the same
        indices.append(i)
    
    return indices

# Assuming 'df' and 'num_classes' are defined
max_window_size = 7
train_indices = generate_indices(train_df, max_window_size)
test_indices = generate_indices(test_df, max_window_size)

print('train df shape: ', train_df.shape)
print('test df shape: ', test_df.shape)
print('Train indices: ', len(train_indices))
print('Test indices: ', len(test_indices))

trainset = SequentialDataset(df=train_df, indices=train_indices, max_window_size=max_window_size)
testset = SequentialDataset(df=test_df, indices=test_indices, max_window_size=max_window_size)

train df shape:  (1462, 7)
test df shape:  (114, 7)
Train indices:  1456
Test indices:  108


In [9]:
train_df

,meantemp,humidity,wind_speed,meanpressure,year,month,day
0,0.122271,1.001431,-1.079544,0.531494,0.0,1,1
1,0.042795,1.344778,-0.562473,0.691120,0.0,1,2
2,0.035662,1.115880,-0.275598,0.755969,0.0,1,3
3,0.081514,0.398665,-0.865544,0.643731,0.0,1,4
4,0.000000,1.108250,-0.437544,0.593848,0.0,1,5
...,...,...,...,...,...,...,...
1457,0.342890,0.248056,-0.463948,0.523903,3.0,12,28
1458,0.282387,1.155120,-0.038462,0.624134,3.0,12,29
1459,0.247453,1.237959,0.007808,0.698959,3.0,12,30
1460,0.276718,1.115880,0.191443,0.563918,3.0,12,31


<a id="3"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #427bf5'>3.</b> Modeling</b></h1>

In [10]:
from tools.setting.data_config import DataConfig
from tools.setting.ml_params import MLParameters
from trainer_hub import TrainerHub

data_config = DataConfig(dataset_name = 'daily-delhi-climate', task_type='regression', obs_shape=[num_features], label_size=1)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(core_model = 'gpt', encoder_model = 'none')

ml_params.num_epoch = 100
ml_params.model.core_config.dropout = 0.1
ml_params.optimization.decay_rate_100k = 0.001

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=True) 

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jeongyoong. Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (.\..\saved\daily-delhi-climate\causal_learning)... Done. 0.0s


In [11]:
trainer_hub.train(trainset, testset)

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Iterations:   0%|          | 0/22 [00:00<?, ?it/s]

Iterations:   0%|          | 0/22 [00:00<?, ?it/s]

Iterations:   0%|          | 0/22 [00:00<?, ?it/s]

[2/100][6/22][Time 5.59]
Unified LR across all optimizers: 0.0001992966486266836
--------------------Training Metrics--------------------
Trainer:  gpt
Inf: 0.0186	Gen: 2.0039	Rec: 2.0041	E: 0.0184	R: 0.0187	P: 3.9894
--------------------Test Metrics------------------------
mse: 15.6113
mae: 3.8997
r2: -402.1982



...

Iterations:   0%|          | 0/22 [00:00<?, ?it/s]

Iterations:   0%|          | 0/22 [00:00<?, ?it/s]

[97/100][16/22][Time 4.91]
Unified LR across all optimizers: 0.0001723852073815896
--------------------Training Metrics--------------------
Trainer:  gpt
Inf: 0.0054	Gen: 0.2514	Rec: 0.2522	E: 0.0046	R: 0.0062	P: 0.4982
--------------------Test Metrics------------------------
mse: 0.0122
mae: 0.0849
r2: 0.6845



Iterations:   0%|          | 0/22 [00:00<?, ?it/s]

Iterations:   0%|          | 0/22 [00:00<?, ?it/s]